# Create targets for WCEP
sklearn does not work in the VM currently 

In [105]:
import json, gzip
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch


def read_jsonl_gz(path):
    with gzip.open(path) as f:
        for l in f:
            yield json.loads(l)

vectorizer = TfidfVectorizer()
data = list(read_jsonl_gz(f'../../data/raw/wcep/train.jsonl.gz'))

targets = []
for cluster in data:
    articles = cluster['articles']
    text = [a['text'] for a in articles][:8]
    X = vectorizer.fit_transform(text)
    targets.append(cosine_similarity(X).sum(1).argmax())

In [106]:
torch.save(targets, '../../data/train_targets_8.pt')